<a href="https://colab.research.google.com/github/amankumar1906/Covid-Detection-using-CNN-and-Genetic-Algorithm/blob/main/ga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from random import random, choice

import os
import shutil
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import load_model
from keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

In [ ]:
root_dir = 'COVID-19_Radiography_Dataset'
source_dirs = ['Normal', 'Viral Pneumonia', 'COVID']
batch_size_val = 16  # how many to process together
steps_per_epoch_val = 10
epochs_val = 30
imageDimesions = (180, 180, 1)
target_size_val = (180, 180)
validationRatio = 0.2 # if 1000 images 20% of remaining 800 will be 160 for validation
noOfClasses = 3
stride_value = 2

In [ ]:
POPULATION_SIZE = 12
GENERATIONS = 20
MUTATION_RATE = 0.3

population = []

pool = {
    'kernel-size': [(3,3),(5,5)],
    'dropout': [0.2,0.1,0.3],
    'learning-rate': [0.001,0.01,0.005],
    'filter': [64,32],
    'activation': ['relu',"sigmoid",'tanh']
}

def initPopulation():
    population.clear()
    for i in range(POPULATION_SIZE):
        population.append({
            'score': 0,
            'kernel-size': choice(pool['kernel-size']),
            'dropout': choice(pool['dropout']),
            'learning-rate': choice(pool['learning-rate']),
            'filter': choice(pool['filter']),
            'activation': choice(pool['activation'])
        })

def buildModel(gene): #U
    no_Of_Nodes = no_Of_Filters = gene['filter']
    size_of_Filter = gene['kernel-size']
    activation = gene['activation']
    dropout = gene['dropout']
    learningRate = gene['learning-rate']

    size_of_pool = (2,2)
    stride_value = 2
    noOfClasses = 3
    imageDimesions = (180,180,1)

    model = Sequential()
    model.add((Conv2D(no_Of_Filters, size_of_Filter, input_shape = imageDimesions, activation=activation)))
    model.add((Conv2D(no_Of_Filters, size_of_Filter, activation=activation)))
    model.add(MaxPooling2D(pool_size = size_of_pool, strides=stride_value)) # DOES NOT EFFECT THE DEPTH/NO OF FILTERS
    model.add(Dropout(dropout))
    model.add((Conv2D(no_Of_Filters, size_of_Filter, activation=activation)))
    model.add((Conv2D(no_Of_Filters, size_of_Filter, activation=activation)))
    model.add(MaxPooling2D(pool_size=size_of_pool, strides=stride_value))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(no_Of_Nodes,activation=activation))
    model.add(Dense(32, activation=activation))
    model.add(Dropout(0.5))
    model.add(Dense(noOfClasses,activation='softmax')) # OUTPUT LAYER
    
    model.compile(Adam(lr=learningRate),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

import json,pickle

def save(gen,data):
    with open('weights/gen'+str(gen)+'.json','w') as f:
        json.dump(data,f)

In [ ]:
def rankSelection(population,noPop=POPULATION_SIZE):
    population = sorted(population, key=lambda x: x['score'])
    scores.append(population[-1]['score'])
    noHalfPopulation = noPop//2 
    halfPopulation = population[noHalfPopulation:]
    return halfPopulation

def tournamentSelection(selectedRank,tSize):
    best = None
    for i in range(0,tSize):
        ind = choice(selectedRank)
        if best == None:
            best = ind
        elif best == ind:
            ind = choice(selectedRank)

        if best['score'] < ind['score']:
            best = ind

    return best

def crossover(p1,p2): 
    crossoverPoint = len(p1)//2
    keys = list(pool.keys())
    crossoverPoint = random.randint(0,len(keys))

    c1, c2 = {}, {}
    for i in range(len(keys)):
        if i < crossoverPoint:
            c1[keys[i]] = p1[keys[i]]
            c2[keys[i]] = p2[keys[i]]
        else:
            c1[keys[i]] = p2[keys[i]]
            c2[keys[i]] = p1[keys[i]]

    return c1, c2

def mutation (c1,c2):
    rate = round(random.random(),2)
    
    if rate <= MUTATION_RATE:
        idx = choice(list(pool.keys()))
        while idx == 'score':
            idx = choice(list(pool.keys()))

        c1[idx] = choice(pool[idx])
        c2[idx] = choice(pool[idx])
        
    return c1,c2


def calculateFitness(population):
    global bestParam
    for i in range(len(population)):
        model = buildModel(population[i])
        history = model.fit_generator(train_generator,epochs=epochs_val,steps_per_epoch=steps_per_epoch_val,verbose=1)
        population[i]['score'] = np.max(history.history['accuracy'])
        if bestParam['score'] < population[i]['score']:
            bestParam = population[i]

In [ ]:
def preprocessing(img): #U
    img = img/255                                   # TO NORMALIZE VALUES BETWEEN 0 AND 1 INSTEAD OF 0 TO 255
    return img

datagen = ImageDataGenerator(preprocessing_function = preprocessing)

train_generator = datagen.flow_from_directory("../input/radiography",
                                              subset = 'training',
                                              color_mode = "grayscale",
                                              target_size = target_size_val,
                                              batch_size = batch_size_val)

Found 300 images belonging to 3 classes.


In [ ]:
mkdir weights

In [ ]:
scores = []
bestParam = {"score":-1}

initPopulation()
print("init done")
for gen in range(0,GENERATIONS):
    print("Calculating fitness for Gen " + str(gen) + "\n")

    calculateFitness(population)
    to_save = {
        'bestParam': bestParam,
        'population': population
    }
    save(gen,to_save)
    selectedRank = rankSelection(population)
    print("\nGeneration {}: {}\n".format(gen,scores[-1]))

    if selectedRank[-1]['score'] > 0.95:
        # stopping condition
        print("Early stopping")
        break

    else:
        child = [] 
        tSize = 2 # this indicate 2-way tournament
        numberOfMating = len(selectedRank)//2
        for i in range (0,numberOfMating):
            parent1 = tournamentSelection(selectedRank,tSize)
            parent2 = tournamentSelection(selectedRank,tSize)
            while parent2 is parent1:
                parent2 = tournamentSelection(selectedRank,tSize)
            c1,c2 = crossover(parent1,parent2)
            c1,c2 = mutation(c1,c2)
            child.append(c1)
            child.append(c2)

        population.clear()
        population = selectedRank + child

init done
Calculating fitness for Gen 0



2021-11-21 11:32:18.447642: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 11:32:18.562985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 11:32:18.563802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 11:32:18.566242: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/30


2021-11-21 11:32:22.501022: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


10/10 [==============================] - 9s 171ms/step - loss: 25.2072 - accuracy: 0.3333
Epoch 2/30
10/10 [==============================] - 1s 77ms/step - loss: 1.1020 - accuracy: 0.3063
Epoch 3/30
10/10 [==============================] - 1s 63ms/step - loss: 1.0997 - accuracy: 0.3125
Epoch 4/30
10/10 [==============================] - 1s 58ms/step - loss: 1.0996 - accuracy: 0.2885
Epoch 5/30
10/10 [==============================] - 1s 57ms/step - loss: 1.0998 - accuracy: 0.3333
Epoch 6/30
10/10 [==============================] - 1s 57ms/step - loss: 1.0977 - accuracy: 0.3846
Epoch 7/30
10/10 [==============================] - 1s 71ms/step - loss: 1.0994 - accuracy: 0.3063
Epoch 8/30
10/10 [==============================] - 1s 57ms/step - loss: 1.0993 - accuracy: 0.3141
Epoch 9/30
10/10 [==============================] - 1s 78ms/step - loss: 1.0986 - accuracy: 0.3438
Epoch 10/30
10/10 [==============================] - 1s 72ms/step - loss: 1.0988 - accuracy: 0.3526
Epoch 11/30
10/10 

In [ ]:
print(bestParam)

{'kernel-size': (3, 3), 'dropout': 0.2, 'learning-rate': 0.001, 'filter': 32, 'activation': 'relu', 'score': 0.9679487347602844}


In [ ]:
for gene in population:
    print(gene)

{'kernel-size': (5, 5), 'dropout': 0.2, 'learning-rate': 0.001, 'filter': 32, 'activation': 'relu', 'score': 0.7756410241127014}
{'kernel-size': (3, 3), 'dropout': 0.2, 'learning-rate': 0.001, 'filter': 64, 'activation': 'relu', 'score': 0.8333333134651184}
{'kernel-size': (5, 5), 'dropout': 0.2, 'learning-rate': 0.001, 'filter': 32, 'activation': 'relu', 'score': 0.78125}
{'kernel-size': (3, 3), 'dropout': 0.2, 'learning-rate': 0.001, 'filter': 64, 'activation': 'relu', 'score': 0.9038461446762085}
{'kernel-size': (3, 3), 'dropout': 0.2, 'learning-rate': 0.001, 'filter': 32, 'activation': 'relu', 'score': 0.84375}
{'kernel-size': (5, 5), 'dropout': 0.2, 'learning-rate': 0.001, 'filter': 32, 'activation': 'relu', 'score': 0.7948718070983887}
{'kernel-size': (3, 3), 'dropout': 0.2, 'learning-rate': 0.001, 'filter': 32, 'activation': 'relu', 'score': 0.9679487347602844}
{'kernel-size': (3, 3), 'dropout': 0.2, 'learning-rate': 0.001, 'filter': 64, 'activation': 'relu', 'score': 0.89999997